In [2]:
import sys
from tqdm import tqdm
sys.version

'3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]'

In [3]:
import re

In [4]:
def prepocess_line(line):
    # 僅僅挑出中文字元，並且斷開不連續的中文字
    # YOUR CODE HERE
    pattern = r'[\u4e00-\u9fff]+'
    segments = re.findall(pattern, line)
    # END YOUR CODE
    return segments

In [5]:
segments = []
with open('./wiki_zh_small.txt',encoding="utf-8") as fr:
    for line in fr.readlines():
        segments += prepocess_line(line)

In [6]:
prepocess_line('“英語”一詞源於遷居英格蘭的日耳曼部落盎格魯（），而“盎格魯”得名於')  
# 應該為：['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

In [7]:
segments[:10]

['英語',
 '英語英語',
 '又稱爲英文',
 '是一種西日耳曼語言',
 '誕生於中世紀早期的英格蘭',
 '如今具有全球通用語的地位',
 '英語',
 '一詞源於遷居英格蘭的日耳曼部落盎格魯',
 '而',
 '盎格魯']

Ngram
一開始要先計算字詞出現的次數

In [10]:
from collections import Counter
#Counter 的 key 是要計數的 item，value 保存的是個數

class Counters:
    def __init__(self, n):
        self.n = n
        self.counters = [Counter() for _ in range(n + 1)]  # 分別代表計算0、1、...個字的出現次數
        # _ means Ignore the index

    def fit(self, segments):
        # 因為 self.counters 分別代表計算0、1、...個字的出現次數
        # 請在此實作利用 segments 以及函式 _skip 來統計次數

        for i in range(1, 1 + self.n):
        # i = 1 ~ 第 n+1 個字
            for seg in tqdm(segments):
                self.counters[i] += Counter(self._skip(seg,i))
                # seg中每個字出現次數 = Counter(self._skip(seg, i))
                
        count = sum(dict(self.counters[1]).values()) # 計算字的總數
        self.counters[0] = Counter({'': count}) # 新增一欄來計算總數
        
        
    def __getitem__(self, k):
        return self.counters[k]

    def _skip(self, segment, n):
        assert n > 0 #指的是程式進行到某個時間點，斷定其必然是某種狀態。這裡指必須是大於 0 的正數
        if len(segment) < n:
            return []
        shift = n - 1
        for i in range(len(segment) - shift):
            yield segment[i:i+shift+1] #丟出值並暫時停止

In [11]:
counters = Counters(n=3)
counters.fit(segments)

100%|████████████████████████████████████████████████████████████████████████████| 48767/48767 [13:11<00:00, 61.59it/s]


In [12]:
counters[0]

Counter({'': 371373})

In [13]:
counters[1]['英']

420

In [14]:
class Ngram:
    def __init__(self, n, counters):
        assert n <= counters.n
        self.n = n
        self.major_counter = counters[n]
        self.minor_counter = counters[n-1]

    def predict_proba(self, prefix='', top_k=5):
        assert len(prefix) >= self.n - 1
        # 使用 Ngram 的公式計算出下一個字出現的機率
        # 輸出為機率與字的tuple列表，詳見下方輸出範例
        if self.n > 1:
            pre = prefix[-(self.n - 1):]
        else:
            pre = ''
            
        count_pre = self.minor_counter[pre]
        probs = []
        for k, count in dict(self.major_counter).items():
            if k.startswith(pre):
                prob = count / count_pre
                probs.append((prob, k[-1]))
        
        sorted_probs = sorted(probs, reverse=True)

        return sorted_probs[:top_k] if top_k > 0 else sorted_probs

    def get_proba_dict(self, prefix=''):
        return {word: prob for prob, word in self.predict_proba(prefix, top_k=-1)}

In [15]:
unigram = Ngram(1, counters)

In [16]:
unigram.predict_proba('我思')

[(0.035732269174118744, '的'),
 (0.012927703414087723, '國'),
 (0.010620050461395955, '中'),
 (0.009984570768472667, '在'),
 (0.009852627950874188, '一')]

In [17]:
bigram = Ngram(2, counters)
trigram = Ngram(3, counters)

使用Ngram來建立第一版選字系統

In [22]:
class ChineseWordRecommenderV1:
    def __init__(self, unigram, bigram, trigram):
        self.unigram = unigram
        self.bigram = bigram
        self.trigram = trigram
    
    def predict_proba(self, prefix='', top_k=5):
        # 使用Ngram來建立選字系統
        if not prefix:
            return self.unigram.predict_proba(prefix, top_k)
        elif len(prefix) == 1:
            return self.bigram.predict_proba(prefix, top_k)
        else:
            return self.trigram.predict_proba(prefix, top_k)

In [24]:
model = ChineseWordRecommenderV1(unigram, bigram, trigram)

In [25]:
probs = model.predict_proba('我思', top_k=10)
probs

[(0.75, '故'), (0.25, '維')]

In [27]:
!pip install -U pip
!pip install -q ipywidgets

  Using cached pip-20.3.4-py2.py3-none-any.whl (1.5 MB)
  Using cached pip-20.3.3-py2.py3-none-any.whl (1.5 MB)


DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [33]:
import ipywidgets as widgets
from IPython.display import display

text = widgets.Textarea()
label = widgets.Label()
display(label, text)

def func(change):
    probs = model.predict_proba(change.new, top_k=10)
    label.value = ' ' + '\t'.join([word for prob, word in probs])

text.observe(func, names='value')